# MuchLearningSuchWow - LSTM - Testing

This notebook contains the code we used to test our LSTM network. The training code is based primarily on [this kernel](https://www.kaggle.com/bountyhunters/baseline-lstm-with-keras-0-7).

### Imports & Data Paths

In [ ]:
import numpy as np
import pandas as pd
import pickle
import time
import keras

import keras.backend as K
import matplotlib.pyplot as plt

In [ ]:
inputPath = "input/m5-forecasting-accuracy/"
outputPath = "output/"
modelPath = "models/"
submissionPath = "submissions/"

### Constants

In [ ]:
timesteps = 14 # Number of previous days that will be used to predict the next day

# Add rolling means?
add_rollingMeans = False

### Loading Data

In [ ]:
with open(outputPath + "/preprocessed_train_valid_data.pkl", "rb") as f:
    df_train_valid = pickle.load(f)
with open(outputPath + "/additional_features_testing.pkl", "rb") as f:
    additional_features_valid, additional_features_eval = pickle.load(f)
with open(outputPath + "/scalers.pkl", "rb") as f:
    valid_scaler, eval_scaler = pickle.load(f)
with open(outputPath + "/item_data.pkl", "rb") as f:
    item_data = pickle.load(f)

In [ ]:
df_sample_submission = pd.read_csv(inputPath + "/sample_submission.csv")

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

valid_model = keras.models.load_model(modelPath + "/lstm_model_valid", 
                                      custom_objects = {'root_mean_squared_error' : root_mean_squared_error})
eval_model = keras.models.load_model(modelPath + "/lstm_model_eval", 
                                     custom_objects = {'root_mean_squared_error' : root_mean_squared_error})

### Testing

In [ ]:
def compute_rolling_means(test_data, predictions):
    # Compute the next row of rolling means (this implementation works because input_data contains more than 7 days)
    test_data = np.squeeze(test_data)
    df_test_pred = pd.DataFrame(np.concatenate([test_data[:,:30490], predictions], axis = 0))
    rolling_mean = pd.DataFrame(df_test_pred.rolling(7).mean())
    rolling_mean = rolling_mean.fillna(0)

    return rolling_mean[-1:]

In [ ]:
def test_model(model, timesteps, input_data, scaler, nr_days_to_predict, additional_features):
    # Scale and convert input data so that it can be fed into the model
    inputs = scaler.transform(input_data)
    X_test = np.array([inputs])
    
    # Predict sales for the next nr_days_to_predict days
    predictions = []
    for j in range(0, nr_days_to_predict):
        feature_shape = 30490 + additional_features.shape[1]
        if(add_rollingMeans): # If rolling means are present, feature_shape is 30490 + # additional features + 30490
            feature_shape += 30490
        model_input = np.append(np.expand_dims(item_data, 0), 
                                X_test[:,-timesteps:,:].reshape(1, timesteps, feature_shape), axis = 1)
        predicted_sales = model.predict(model_input)
        to_stack = [np.array(predicted_sales), additional_features.iloc[[j]]]
        if(add_rollingMeans): # If rolling means are required, compute them and add them to model_output
            rolling_means = compute_rolling_means(X_test, predicted_sales)
            to_stack.append(rolling_means)
        model_output = np.column_stack(tuple(to_stack))
        model_output_expanded = np.expand_dims(model_output, 0)
        X_test = np.append(X_test, model_output_expanded, axis = 1)
        predicted_sales = scaler.inverse_transform(model_output)[:,0:30490]
        predictions.append(predicted_sales)
    
    return predictions

In [ ]:
predictions_valid = test_model(valid_model, timesteps, df_train_valid[-28-timesteps:-28], 
                               valid_scaler, 28, additional_features_valid)
predictions_eval = test_model(eval_model, timesteps, df_train_valid[-timesteps:], 
                              eval_scaler, 28, additional_features_eval)

### Writing Submission File

In [ ]:
submission_valid = pd.DataFrame(data=np.array(predictions_valid).reshape(28,30490))
submission_valid = submission_valid.T
submission_eval = pd.DataFrame(data=np.array(predictions_eval).reshape(28,30490))
submission_eval = submission_eval.T

In [ ]:
submission = pd.concat((submission_valid, submission_eval), ignore_index=True)
idColumn = df_sample_submission[["id"]]
submission[["id"]] = idColumn  
cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]
colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]
submission.columns = colsdeneme
submission.to_csv(submissionPath + "/submission.csv", index=False)